# Proyecto 2 modelos

In [2]:
import pandas as pd
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import ndimage
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import cv2
from PIL import Image

In [25]:
import torch
import torch.nn as nn
import torchvision.models as models

# Define la arquitectura del modelo combinado
class CombinedModel(nn.Module):
    def __init__(self, vgg_output_size, gru_hidden_size, gru_num_layers, num_classes):
        super(CombinedModel, self).__init__()
        self.vgg16 = models.vgg16(pretrained=True)
        self.vgg16.features[28] = nn.AvgPool2d(kernel_size=7, stride=1)  # Cambia el stride de la última capa de pooling
        self.vgg16.classifier = nn.Identity()  # Desactiva las capas clasificadoras

        self.gru = nn.GRU(vgg_output_size, gru_hidden_size, num_layers=gru_num_layers, batch_first=True)
        self.fc = nn.Linear(gru_hidden_size, num_classes)

    def forward(self, x):
        # Pasa cada imagen a través de VGG16 y obtén los feature maps
        vgg_outputs = []
        for image in x:
            vgg_output = self.vgg16(image)
            vgg_output = vgg_output.view(vgg_output.size(0), -1, vgg_output.size(1))
            vgg_outputs.append(vgg_output)
        
        # Concatena los feature maps de las imágenes a lo largo de la dimensión del tiempo (secuencia)
        gru_input = torch.cat(vgg_outputs, dim=0)  # Concatena en la dimensión 0 (batch_size x num_images, 512, height, width)

        # Reorganiza la entrada para que cada imagen sea una "secuencia" para la GRU
        gru_input = gru_input.view(-1, len(vgg_outputs), vgg_output_size)  # (batch_size, num_images, 512)

        # Pasa la secuencia de entrada a través de GRU
        gru_output, _ = self.gru(gru_input)

        # Pasa la última salida de la GRU a través de una capa completamente conectada
        output = self.fc(gru_output[:, -1, :])
        return output

# Parámetros del modelo
vgg_output_size = 512  # Tamaño de la salida de la VGG16
gru_hidden_size = 128  # Tamaño del estado oculto de la GRU
gru_num_layers = 2  # Número de capas en la GRU
num_classes = 7  # Reemplaza con el número de clases en tu problema

# Crea una instancia del modelo combinado
model_combined = CombinedModel(vgg_output_size, gru_hidden_size, gru_num_layers, num_classes)


c:\Users\Daniel\Main\UVG\Semestre VIII\Data science\Proyecto2\myenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Daniel\Main\UVG\Semestre VIII\Data science\Proyecto2\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
from torchvision import transforms

# Define la función para preprocesar las imágenes para VGG16 y CombinedModel
def preprocess_images_for_combined_model(image_set):
    # Transformaciones para preprocesar las imágenes para VGG16
    transform = transforms.Compose([
        transforms.ToPILImage(),  # Convertir a objeto PIL (es necesario para la VGG16)
        transforms.Resize((224, 224)),  # Cambiar el tamaño a 224x224 (tamaño de entrada de la VGG16)
        transforms.ToTensor(),  # Convertir a tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización requerida por VGG16
    ])

    preprocessed_images = []
    cont = 0
    for image in image_set:
        preprocessed_image = transform(image)  # Aplica las transformaciones
        preprocessed_images.append(preprocessed_image)
        cont += 1
        if cont == 50:
            break
    return torch.stack(preprocessed_images)  # Apila las imágenes en un tensor

# Preprocesa las imágenes para VGG16 y CombinedModel
preprocessed_images_tensor = preprocess_images_for_combined_model(image_set)
print(f'Imagenes preprocesadas para VGG16: {preprocessed_images_tensor.shape}')

# Ahora, puedes pasar estas imágenes preprocesadas al modelo CombinedModel
output = model_combined(preprocessed_images_tensor)
print(f'Salida del modelo CombinedModel: {output.shape}')

Imagenes preprocesadas para VGG16: torch.Size([50, 3, 224, 224])
Salida del modelo CombinedModel: torch.Size([49, 7])


In [4]:
def read_images_from_folder(folder_path):
    image_set = []  # Lista para almacenar las imágenes

    # Obtén la lista de archivos en la carpeta
    files = os.listdir(folder_path)

    for file in files:
        # Verifica si el archivo es una imagen
        if file.endswith(('png', 'jpg', 'jpeg', 'gif')):
            # Carga la imagen y conviértela a un arreglo numpy
            image_path = os.path.join(folder_path, file)
            image = Image.open(image_path)
            image = np.array(image)

            # Agrega la imagen al conjunto
            image_set.append(image)

    return np.array(image_set)

# Ruta de la carpeta que contiene las imágenes
folder_path = "imagenes"

# Lee las imágenes desde la carpeta y obtén el conjunto de imágenes
image_set = read_images_from_folder(folder_path)

In [10]:



# Crear una muestra de entrenamiento con el conjunto de imágenes y la etiqueta
train_sample = {'image_set': image_set, 'label': 1}

# Convertir las imágenes a tensores de PyTorch y agregar una dimensión adicional para representar el batch size (1 en este caso)
images_tensor = torch.tensor(train_sample['image_set'], dtype=torch.float32).unsqueeze(0)
print(f'Imagenes: {images_tensor.shape}')

# Convertir la etiqueta a tensor de PyTorch
label_tensor = torch.tensor(train_sample['label'], dtype=torch.long)

# Define el modelo, la función de pérdida y el optimizador
model_combined = CombinedModel(vgg_output_size, gru_hidden_size, gru_num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_combined.parameters(), lr=0.001)

# Entrenamiento del modelo
model_combined.train()
optimizer.zero_grad()
outputs = model_combined(images_tensor)
loss = criterion(outputs, label_tensor.unsqueeze(0))  # Se agrega una dimensión para la etiqueta
loss.backward()
optimizer.step()

print(f'Loss: {loss.item()}')

Imagenes: torch.Size([1, 436, 512, 512, 3])
torch.Size([436, 512, 512, 3])


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[436, 512, 512, 3] to have 3 channels, but got 512 channels instead